In [7]:
import pandas as pd
from sklearn.cluster import KMeans
import tensorflow as tf
import numpy as np
import joblib

In [8]:
# Load student and teacher data from CSV files
students_df = pd.read_csv('students.csv')
teachers_df = pd.read_csv('teachers.csv')

In [9]:
# Combine interests and CGPA into a single feature vector for clustering
student_features = students_df[['CGPA']].values

In [10]:
# Define the number of clusters (teachers)
num_clusters = len(teachers_df)

In [11]:
# Create a KMeans clustering model
kmeans = KMeans(n_clusters=num_clusters, n_init=10, random_state=0).fit(student_features)

In [12]:
# Save the KMeans model to a file
joblib.dump(kmeans, 'kmeans_model.pkl')

['kmeans_model.pkl']

In [13]:
# Assign each student to a cluster (teacher)
students_df['Cluster'] = kmeans.labels_

In [14]:
# Initialize teacher assignments
teacher_assignments = {teacher: [] for teacher in teachers_df['Teacher_name']}

In [15]:
# Iterate through students and assign them to the nearest matching teacher
for _, student in students_df.iterrows():
    cluster_label = student['Cluster']
    teacher_name = teachers_df.iloc[cluster_label]['Teacher_name']
    student_name = student['Student_name']
    cgpa = student['CGPA']
    teacher_assignments[teacher_name].append((student_name, cgpa))

In [16]:
# Print the assignments with teacher name, student name, and CGPA
for teacher, students in teacher_assignments.items():
    print(f"Teacher: {teacher}")
    for student, cgpa in students:
        print(f"  Student: {student}, CGPA: {cgpa}")

Teacher: Pankaj
  Student: Olivia Bennett, CGPA: 9.4
  Student: Liam Anderson, CGPA: 9.2
  Student: Emma Parker, CGPA: 9.5
  Student: Noah Harrison, CGPA: 9.6
  Student: Elijah Brooks, CGPA: 9.4
  Student: Scarlett Adams, CGPA: 9.4
Teacher: Amit
  Student: Aiden Carter, CGPA: 7.0
  Student: Jackson Walker, CGPA: 7.2
  Student: Abigail Simmons, CGPA: 7.3
  Student: Evelyn Taylor, CGPA: 7.2
  Student: Chloe Moore, CGPA: 7.6
Teacher: Akshay
  Student: Ava Mitchell, CGPA: 8.0
  Student: Ethan Turner, CGPA: 8.4
  Student: Sophia Jenkins, CGPA: 8.2
  Student: Mason Foster, CGPA: 8.1
  Student: Oliver Rivera, CGPA: 8.2
  Student: Harper Gray, CGPA: 8.3
  Student: Samuel Davis, CGPA: 7.9
  Student: Lily Johnson, CGPA: 8.1
  Student: Daniel Robinson, CGPA: 7.8
Teacher: Sachin
  Student: Mia Nelson, CGPA: 6.5
  Student: Lucas Cooper, CGPA: 6.3
  Student: Charlotte Hayes, CGPA: 6.2
  Student: Caleb Smith, CGPA: 6.3
  Student: Henry Martinez, CGPA: 6.7
Teacher: Ranjodh
  Student: Isabella Murphy, 

In [17]:
# Convert the KMeans model to TensorFlow Lite
kmeans_model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(student_features.shape[1],)),
    tf.keras.layers.Dense(num_clusters, activation='softmax')
])

In [18]:
kmeans_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
kmeans_model.fit(student_features, kmeans.labels_, epochs=10)

Epoch 1/10
1/1 [==============================] - 0s 429ms/step - loss: 4.9167
Epoch 2/10
1/1 [==============================] - 0s 12ms/step - loss: 4.9065
Epoch 3/10
1/1 [==============================] - 0s 10ms/step - loss: 4.8965
Epoch 4/10
1/1 [==============================] - 0s 16ms/step - loss: 4.8864
Epoch 5/10
1/1 [==============================] - 0s 17ms/step - loss: 4.8765
Epoch 6/10
1/1 [==============================] - 0s 19ms/step - loss: 4.8666
Epoch 7/10
1/1 [==============================] - 0s 19ms/step - loss: 4.8568
Epoch 8/10
1/1 [==============================] - 0s 19ms/step - loss: 4.8471
Epoch 9/10
1/1 [==============================] - 0s 8ms/step - loss: 4.8374
Epoch 10/10
1/1 [==============================] - 0s 8ms/step - loss: 4.8278


In [19]:
# Convert the KMeans model to TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(kmeans_model)
tflite_model = converter.convert()

# Save the TensorFlow Lite model to a file
with open('student_clustering_model.tflite', 'wb') as f:
    f.write(tflite_model)


INFO:tensorflow:Assets written to: C:\Users\itars\AppData\Local\Temp\tmpfffrjkrd\assets


INFO:tensorflow:Assets written to: C:\Users\itars\AppData\Local\Temp\tmpfffrjkrd\assets
